In [10]:
import pandas as pd
import numpy as np
from icecream import ic

### 字符串处理

In [11]:
s_abc = pd.Series(["a", "b", "c"])
print((s_abc.str.upper()))

0    A
1    B
2    C
dtype: object


In [12]:
s = pd.Series([u"北京", u"北京市", u"北京地区"]) #字符串
s_utf8=s.str.encode("utf-8") #utf-8二进制
s_unicode = s_utf8.str.decode("utf-8") #字符串
s_gb2312 = s_unicode.str.encode("gb2312") #gb2312二进制

print(s_utf8.str.len(),s_unicode.str.len(),s_gb2312.str.len(),sep="\n")
#len函数作用于二进制和字符串的效果是不同的
print("北京".encode()) #默认unicode编码
print("北京".encode("utf-8"))
print("北京".encode("gb2312"))

0     6
1     9
2    12
dtype: int64
0    2
1    3
2    4
dtype: int64
0    4
1    6
2    8
dtype: int64
b'\xe5\x8c\x97\xe4\xba\xac'
b'\xe5\x8c\x97\xe4\xba\xac'
b'\xb1\xb1\xbe\xa9'


In [13]:
print((s_unicode.str[:2]))

0    北京
1    北京
2    北京
dtype: object


In [14]:
print((s_unicode + "-" + s_abc * 2)) #df的加法针对index相同


0      北京-aa
1     北京市-bb
2    北京地区-cc
dtype: object


In [15]:
print((s_unicode.str.cat(s_abc, sep="-")))#str连接函数

0      北京-a
1     北京市-b
2    北京地区-c
dtype: object


In [16]:
print((s_unicode.str.len().astype(str)))

0    2
1    3
2    4
dtype: object


In [17]:
s = pd.Series(["a|bc|de", "x|xyz|yz"])
s_list = s.str.split("|")
s_comma = s_list.str.join(",")
%C s;s_list;s_comma

      s              s_list          s_comma   
-------------  -----------------  -------------
0     a|bc|de  0     [a, bc, de]  0     a,bc,de
1    x|xyz|yz  1    [x, xyz, yz]  1    x,xyz,yz
dtype: object  dtype: object      dtype: object


In [18]:
s_list.str[1] #series的元素为列表时，可以经由str进行索引或切片
s_list.str[:3]

0     [a, bc, de]
1    [x, xyz, yz]
dtype: object

In [19]:
pd.DataFrame(s_list.tolist(), columns=["A", "B", "C"]) #使用tolist方法将series转换为列表，再转换为df

,A,B,C
0,a,bc,de
1,x,xyz,yz


In [20]:
#使用正则表达式方法从series中抽出适当的元素组成df,括号代表group
df_extract1 = s.str.extract(r"(\w+)\|(\w+)\|(\w+)")
df_extract2 = s.str.extract(r"(?P<A>\w+)\|(?P<B>\w+)|") #?P<colname>
print(df_extract1)
print("sep".center(50,"-"))
print(df_extract2)


   0    1   2
0  a   bc  de
1  x  xyz  yz
-----------------------sep------------------------
   A    B
0  a   bc
1  x  xyz


In [21]:
#用read_csv的方法从string变量中读数据
import io
text = """A, B|C|D
B, E|F
C, A
D, B|C
"""
df = pd.read_csv(io.StringIO(text), skipinitialspace=True, header=None) #忽略掉分隔符后面的空格
df

,0,1
0,A,B|C|D
1,B,E|F
2,C,A
3,D,B|C


In [22]:
nodes = df[1].str.split("|") #把数据列1的元素拆分成列表
from_node = df[0].values.repeat(nodes.str.len()) #将第一列数据按照第二列元素列表长度重复
to_node = np.concatenate(nodes) #因为元素list长度不一样，所以不好用ravel进行展平，故使用concatenate进行矩阵合并

df2=pd.DataFrame({"from_node":from_node, "to_node":to_node}) #字典赋值
df2

,from_node,to_node
0,A,B
1,A,C
2,A,D
3,B,E
4,B,F
5,C,A
6,D,B
7,D,C


In [23]:
print((df[1].str.get_dummies(sep="|"))) #每行是否包含某个列标签

   A  B  C  D  E  F
0  0  1  1  1  0  0
1  0  0  0  0  1  1
2  1  0  0  0  0  0
3  0  1  1  0  0  0


In [24]:
df[1].map(lambda s:max(s.split("|"))) #使用map函数针对每个元素做运算

0    D
1    F
2    A
3    C
Name: 1, dtype: object

In [25]:
df_soil = pd.read_csv("Soils.csv", usecols=[2, 3, 4, 6])
print((df_soil.dtypes))

Contour     object
Depth       object
Gp          object
pH         float64
dtype: object


In [26]:
#将文本类型的数据转换为分类变量，可以节省内存
for col in ["Contour", "Depth", "Gp"]:
    df_soil[col] = df_soil[col].astype("category")
print((df_soil.dtypes))

Contour    category
Depth      category
Gp         category
pH          float64
dtype: object


In [27]:
Gp = df_soil.Gp
print((Gp.cat.categories)) #cat名称空间中储存了分类变量的信息

Index(['D0', 'D1', 'D3', 'D6', 'S0', 'S1', 'S3', 'S6', 'T0', 'T1', 'T3', 'T6'], dtype='object')


In [28]:
%C Gp.head(5);Gp.cat.codes.head(5)

                                  Gp.head(5)                                    Gp.cat.codes.head(5)
------------------------------------------------------------------------------  --------------------
0    T0                                                                         0    8              
1    T0                                                                         1    8              
2    T0                                                                         2    8              
3    T0                                                                         3    8              
4    T1                                                                         4    9              
Name: Gp, dtype: category                                                       dtype: int8         
Categories (12, object): ['D0', 'D1', 'D3', 'D6', ..., 'T0', 'T1', 'T3', 'T6']                      


In [33]:
#分类变量分有序和无序，可以使用as_ordered()和as_unordered()进行转换
depth = df_soil.Depth
%C depth.cat.as_ordered().head()

                depth.cat.as_ordered().head()                                           depth.head()                       
--------------------------------------------------------------  -----------------------------------------------------------
0     0-10                                                      0     0-10                                                 
1     0-10                                                      1     0-10                                                 
2     0-10                                                      2     0-10                                                 
3     0-10                                                      3     0-10                                                 
4    10-30                                                      4    10-30                                                 
Name: Depth, dtype: category                                    Name: Depth, dtype: category                               
Categori

In [40]:
#指定分类顺序
contour = df_soil.Contour
categories = ["Slope","Top", "Depression"]
%C contour.cat.reorder_categories(categories, ordered=True).head()

contour.cat.reorder_categories(categories, ordered=True).head()
---------------------------------------------------------------
0    Top                                                       
1    Top                                                       
2    Top                                                       
3    Top                                                       
4    Top                                                       
Name: Contour, dtype: category                                 
Categories (3, object): ['Slope' < 'Top' < 'Depression']       
